<a href="https://colab.research.google.com/github/fcadiz/vespula2/blob/master/SCRIPT_DEFINITIVO_TENSORFLOW_PANCHO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUPER SCRIPT SPANGLISH ENTRENAMIENTO IA RECONOCIMIENTO OBJETOS

## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

Llama a la versión estable de tensorflow 1

In [ ]:
%tensorflow_version 1.x

In [ ]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/fcadiz/vespula2'
 
# Number of training steps.
num_steps = 400000  # 200000
 
# Number of evaluation steps.
num_eval_steps = 150
 
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 5
    },
    'ssd_mobilenet_v3': {
        'model_name': 'ssd_mobilenet_v3_large_coco_2020_01_14',
        'pipeline_file': 'ssdlite_mobilenet_v3_large_320x320_coco.config',
        'batch_size': 40
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}
 
# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'
 
# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']
 
# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']
 
# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Clone the `object_detection_demo` repository or your fork.

In [ ]:
import os
 
%cd /content
 
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
 
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

## Install required packages

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
 
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk lvis protobuf

!pip install lvis

 
!pip install -q Cython contextlib2 pillow lxml matplotlib lvis protobuf
 
!pip install -q pycocotools
 
%cd /content/models/research/
!pip install .
 
%cd /content/models/research/slim
!pip install .
 
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
 
!python object_detection/builders/model_builder_test.py

!pip install -q torchvision

## Prepare `tfrecord` files

Use the following scripts to generate the `tfrecord` files.
```bash
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt
```

In [ ]:
%cd {repo_dir_path}
 
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i /content/vespula2 -o data/annotations/train_labels.csv -l data/annotations
 
# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i /content/vespula2 -o data/annotations/test_labels.csv
 
# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=/content/vespula2/data/images/train --label_map data/annotations/label_map.pbtxt
 
# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=/content/vespula2/data/images/test --label_map data/annotations/label_map.pbtxt
 
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations
 
# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv
 
# Generate `train.record`
!python generate_tfrecord.py --csv_input=/content/vespula2/data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=/content/vespula2/data/images/train --label_map data/annotations/label_map.pbtxt
 
# Generate `test.record`
!python generate_tfrecord.py --csv_input=/content/vespula2/data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=/content/vespula2/data/images/test --label_map data/annotations/label_map.pbtxt

In [ ]:
test_record_fname = '/content/vespula2/data/annotations/test.record'
train_record_fname = '/content/vespula2/data/annotations/train.record'
label_map_pbtxt_fname = '/content/vespula2/data/annotations/label_map.pbtxt'

## Download base model

In [ ]:
%cd /content/models/research
 
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'
model_dir= '/content/models/research/pretrained_model'
 
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
 
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()
 
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

**Agregar aquí el path de Drive en la que se guardarán los resultados parciales del entrenamiento**

In [ ]:
drive_dir= '/content/drive/My Drive/colab_10_11_2020/'

In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

## Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)
#pipeline_fname = os.path.join('/content/drive/My Drive/colab_2_11_2020/', pipeline_file)
 
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re
 
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
 
    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)
 
    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)
 
    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
!cat {pipeline_fname}

Montar drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Elimina la carpeta local de destino en la que se copian los checkpoint parciales o finales del Drive

In [ ]:
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)

## Run Tensorboard(Optional)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = '/content/drive/My Drive/colab_8_11_2020'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Train the model

In [ ]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

se deben guardar los archivos en esa carpeta del drive, y se puede seguir entrenando al aumentar el número de pasos en la programación al comienzo del cuaderno.
Después se echa a correr de nuevo y continúa el entrenamiento, que comienza en el paso anterior.

In [ ]:
# cd /content/vespula2/modelo_definitivo
# cd /content/vespula2
!cd {model_dir}

In [ ]:
!ls {model_dir}

la de abajo es para limpiar la carpeta local en la que se guardará la info que está en el drive

In [ ]:
# limpiar la carpeta local de destino
!rm -rf {'/content/models/research/pretrained_model'}
# os.makedirs('/content/models/research/pretrained_model', exist_ok=True)

para copiar algún estadio intermedio del análisis en un punto x del tiempo de entrenamiento

In [ ]:
%cp -r '{drive_dir}' 'os.path.join(model_dir, 'intermedio_NUM_ULT_CHECK')'

Copiar desde Drive a Local

In [ ]:
%cp -r '{drive_dir}' '{model_dir}'

In [ ]:
!cd {model_dir}

In [ ]:
!ls {model_dir}

In [ ]:
# Legacy way of training(also works).
# !python /content/models/research/object_detection/legacy/train.py --logtostderr --train_dir={model_dir} --pipeline_config_path={pipeline_fname}

## Exporting a Trained Inference Graph
/content/models/research/fine_tuned_model/frozen_inference_graph.pb

In [ ]:
!rm -rf {'/content/models/research/fine_tuned_model'}

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

# Exporting a TFLite compatible Frozen Graph

In [ ]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path='/content/models/research/fine_tuned_model/pipeline.config' \
--trained_checkpoint_prefix={last_model_path} \
--output_directory='./fine_tuned_model' \
--add_postprocessing_op=True

Transformar a detect.tflite

In [ ]:
!toco \
--graph_def_file='/content/models/research/fine_tuned_model/tflite_graph.pb' \
--output_file='/content/models/research/fine_tuned_model/detect.tflite' \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=FLOAT \
--allow_custom_ops

Este es el saved_model.pb y ya está listo

In [ ]:
/content/models/research/fine_tuned_model/saved_model/saved_model.pb

# con este se copian todos los archivos resultantes de regreso al drive

In [ ]:
%cp -r '/content/models/research/fine_tuned_model' 'os.path.join(drive_dir, 'resultados_NUM_ULT_CHECK')'

# Pack Metadata

In [ ]:
!pip install tflite-support

In [ ]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

""" ... """
"""Creates the metadata for an image classifier."""

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "MobileNetV2 SSD OID TF1 300x300 Object Detector"
model_meta.description = ("Permite localizar e identificar avispas, abejas "
                          "y otros insectos. Versión en entrenamiento ")
model_meta.version = "v1"
model_meta.author = "FC"
model_meta.license = ("Apache License. Version 2.0 "
                      "http://www.apache.org/licenses/LICENSE-2.0.")

In [ ]:
# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()

In [ ]:
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(300, 300))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

**cambiar direcciones de los archivos reales**

In [ ]:
labelmap_file = "/content/assets/labelmap.txt"


export_model_path = "/content/assets/detect.tflite"

In [ ]:
import os

In [ ]:
# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probabilidad"
output_meta.description = "Probabilidad de detectar los insectos."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(labelmap_file)
label_file.description = "Etiquetas para los insectos que el modelo puede reconocer."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

In [ ]:
# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta, output_meta, output_meta, output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

In [ ]:

populator = _metadata.MetadataPopulator.with_model_file(export_model_path)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([labelmap_file])
populator.populate()

**cambiar también la dirección de abajo**

In [ ]:

displayer = _metadata.MetadataDisplayer.with_model_file(export_model_path)
export_json_file = os.path.join("/content/assets/", #"detect.json")
                    os.path.splitext(os.path.basename(export_model_path))[0] + ".json")
json_file = displayer.get_metadata_json()
# Optional: write out the metadata as a json file
with open(export_json_file, "w") as f:
  f.write(json_file)

# Descargar video

abrir Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

instalar youtube-dl

In [ ]:
!pip install --upgrade youtube-dl

Definir carpeta en la que se guardarán los videos

In [ ]:
%cd /content/drive/My Drive/training_images/videos/vespula_germanica

Descargar listado de videos, solo agregando los 11 alfanuméricos del nombre separados por un espacio

In [ ]:
!python3 getvideo.py Ons6X1wB7gY fvTlhHIp1rI fs2KOirLwm8 btXkdmEs9Lc MI41doGgKt8 bDuZkbV33lA pNI6BLDGwKA 61uLheWI_Y4 c4dxeKeOOUc bX40kC56ci0 TCMbKCdzEzg B0vNi7oNhq0 urI99LWXJIg a73BZKHEL20 rXCPTN5A6rA 1e1SoyUJ3WA IiQJhEwW1q8 BdymgM_BMRA lUy-dhDcFik n518zRlTBnA IA3Um5JtMqU cpbtQAaO2pA kB7oEVBuADs pLUcYfTjYYo Qrxxw3dV-sc m-JZXwYDVRo AM-vJDaUCAE rBWYUqzXXtY pGUl_3onSn4 uKKyq6LuHfY XGLkis4xyu4 _67Iz3331tg DOa3AXPoU3A WlDN5cIRevQ Hl0XNj93cmg #¿NOMBRE? BnthjmjFOOU 2TmoKH0kP9c YI86A-kcHPU Sqc0wyUx_j0 dcY7XnaBg14 8aSnlG8Zelw 5xdjD3qltto m97M4LTQFGk CwcsZBST3DY h7M-SpKSVKs 9rCoaqiyxIU u3mQJnvYxjg sIDKrFX1dO0 qilS3bZaoG0 Qf0fLsZCSJ4 NL8acsU864g P0ALo5J9RaI 1zecC7qTxjo QnHqy1UDnUc PpU-XgkoNiU urI99LWXJIg 1e1SoyUJ3WA   

revisar el listado de videos descargados

In [ ]:
%ls

mover videos a diferentes carpetas, de acuerdo a la extensión

In [ ]:
%mkdir /content/drive/My\ Drive/training_images/videos/tmp1
%cp -r '/content/drive/My Drive/training_images/videos/vespula_germanica/'*.mkv '/content/drive/My Drive/training_images/videos/tmp1'
%mkdir /content/drive/My\ Drive/training_images/videos/tmp2
%cp -r '/content/drive/My Drive/training_images/videos/vespula_germanica/'*.mp4 '/content/drive/My Drive/training_images/videos/tmp2'
%mkdir /content/drive/My\ Drive/training_images/videos/tmp3
%mv '/content/drive/My Drive/training_images/videos/tmp1' '/content/drive/My Drive/training_images/videos/tmp3'
%mv '/content/drive/My Drive/training_images/videos/tmp2' '/content/drive/My Drive/training_images/videos/tmp3'
%mkdir /content/drive/My\ Drive/training_images/videos/tmp4

Mover los videos desde las carpetas con mkv y mp4 a una carpeta que los agrupa

In [ ]:
%cp -r '/content/drive/My Drive/training_images/videos/tmp3/tmp1/'*.mkv '/content/drive/My Drive/training_images/videos/tmp4'
%cp -r '/content/drive/My Drive/training_images/videos/tmp3/tmp2/'*.mp4 '/content/drive/My Drive/training_images/videos/tmp4'
%cd /content/drive/My Drive/training_images/videos/tmp4

Con estos script se toman los archivos de una carpeta, se ordenan por nombre y se les cambia el nombre de acuerdo al orden alfabético, con un número desd el 0 o el 1

In [ ]:
import os
from os import path
import shutil

Source_Path = '/content/drive/My Drive/training_images/videos/tmp3/tmp1/'
Destination = '/content/drive/My Drive/training_images/videos/tmp3/tmp1/'
#dst_folder = os.mkdir(Destination)


def main():
    for count, filename in enumerate(os.listdir(Source_Path)):
        dst =  "video" + "_" + str(count+1) + ".mkv"

        # rename all the files
        os.rename(os.path.join(Source_Path, filename),  os.path.join(Destination, dst))


# Driver Code
if __name__ == '__main__':
    main()

In [ ]:
import os
from os import path
import shutil

Source_Path = '/content/drive/My Drive/training_images/videos/tmp3/tmp2/'
Destination = '/content/drive/My Drive/training_images/videos/tmp3/tmp2/'
#dst_folder = os.mkdir(Destination)


def main():
    for count, filename in enumerate(os.listdir(Source_Path)):
        dst =  "video" + "_" + str(count+1) + ".mp4"

        # rename all the files
        os.rename(os.path.join(Source_Path, filename),  os.path.join(Destination, dst))


# Driver Code
if __name__ == '__main__':
    main()

condensa todos los videos de una carpeta, de la misma extensión, en un solo video final

In [ ]:
from moviepy.editor import *
import os
from natsort import natsorted

L =[]

for root, dirs, files in os.walk("/content/drive/My Drive/training_images/videos/tmp4"):

    #files.sort()
    files = natsorted(files)
    for file in files:
        if os.path.splitext(file)[1] == '.mp4':
            filePath = os.path.join(root, file)
            video = VideoFileClip(filePath)
            L.append(video)

final_clip = concatenate_videoclips(L)
final_clip.to_videofile("output.mp4", fps=24, remove_temp=False)

In [ ]:
from moviepy.editor import *
import os
from natsort import natsorted

L =[]

for root, dirs, files in os.walk("/content/drive/My Drive/training_images/videos/tmp4"):

    #files.sort()
    files = natsorted(files)
    for file in files:
        if os.path.splitext(file)[1] == '.mkv':
            filePath = os.path.join(root, file)
            video = VideoFileClip(filePath)
            L.append(video)

final_clip = concatenate_videoclips(L)
final_clip.to_videofile("output.mkv", fps=24, remove_temp=False)

# Trabajo edición de video a imagen a etiquetado



cambiar directorio al de los videos compilados

In [ ]:
%cd /content/drive/My Drive/training_images/videos/tmp4

revisar listado

In [ ]:
%ls

extraer frames de los videos a jpg. 0.5 = 1 frame por cada 0.5 segundos, 1 = 1 frame por segundo, 2 = 1 frame por cada 2 segundos

In [ ]:
import cv2
vidcap = cv2.VideoCapture('output.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(str(count-1)+ "mp4" + ".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 2 #//it will capture image in each 1 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
import cv2
vidcap = cv2.VideoCapture('output.mkv')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(str(count-1)+ "mkv" + ".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 2 #//it will capture image in each 1 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

# Anotación automática de las imágenes utilizando un modelo propio

cambiar dirección de descarga del repo. los scripts están hechos para funcionar con esa carpeta de raíz.

In [ ]:
%cd /content

descargar repo a local

In [ ]:
!git clone https://github.com/AlvaroCavalcante/auto_annotate

cambiar carpeta a la que contiene los scripts del object_detection

In [ ]:
%cd /content/models/research/object_detection/utils

cambiar nombre del script que se reemplazará. alternativa a borrarlo

In [ ]:
%mv "visualization_utils.py" "visualization_utils22.py"

cambiar a la carpeta de scripts del repo

In [ ]:
%cd /content/auto_annotate/scripts

mandar script de la repo al object_detection

In [ ]:
%cp -r visualization_utils.py '/content/models/research/object_detection/utils'

mandar script modificado desde mi drive al object_detectiom

In [ ]:
%cp '/content/drive/My Drive/scripts/generate_xml.py' '/content/models/research/object_detection/utils'

cambiar a carpeta de mi drive con el modelo propio y su archivo de etiquetas de objetos

In [ ]:
%cd /content/drive/My Drive/Colab_tensorflow/colab_8_11_2020/resultados_221672

mandar gráfico y etiquetas a la carpeta correspondiente del repo

In [ ]:
%cp frozen_inference_graph.pb label_map.pbtxt '/content/auto_annotate/graphs'

instalar el elementtree para anotar los archivos .xml de cada imagen

In [ ]:
!pip install http://effbot.org/media/downloads/elementtree-1.2.6-20050316.zip

In [ ]:
!pip install elementtree -vvv

limpiar las carpetas de destino, ya que los readme cagan la corrida de los scripts

In [ ]:
import os
!rm -rf {'/content/auto_annotate/xml'}
!rm -rf {'/content/auto_annotate/results'}
!rm -rf {'/content/auto_annotate/images'}
os.makedirs('/content/auto_annotate/xml', exist_ok=True)
os.makedirs('/content/auto_annotate/images', exist_ok=True)
os.makedirs('/content/auto_annotate/results', exist_ok=True)

pasar las imagenes obtenidas de los videos a la carpeta correspondientes del repo

In [ ]:
%cp -r '/content/drive/My Drive/Videos2/'*.jpg '/content/auto_annotate/images'

cambiar el script principal de carpeta, para que corra correctamente

In [ ]:
%mv '/content/auto_annotate/scripts/detection_images.py' '/content/auto_annotate'

cambiar carpeta a la del script principal

In [ ]:
%cd /content/auto_annotate

correr script

In [ ]:
!python3 detection_images.py

mandar los .xml a la carpeta que tiene los resultados de imagenes

In [ ]:
%cp -r '/content/auto_annotate/xml/'*.xml '/content/auto_annotate/results'

copiar los resultados de local a mi drive. **Modificar la carpeta, que se crea automática**. Si ya existe, se crea una subcarpeta y eso no me gusta

In [ ]:
%cp -r '/content/auto_annotate/results' '/content/drive/My Drive/partial_outc5'

**Eliminar las imagenes sin cuadros de detección en la carpeta de arriba** directo en el drive- Chequear que el número de jpg y xml correspondan.

Abajo, se mueven los archivos que tienen cuadros de detección a una carpeta temporal. Estos no coinciden con los xml, entonces se necesita renombrarlos.

In [ ]:
%mkdir /content/auto_annotate/temp
%cp -r '/content/drive/My Drive/partial_outc5/'*.jpg '/content/auto_annotate/temp'
%cd /content/auto_annotate/temp

Renombrar los jpg para que coincidan con los xml. Se envían a la carpeta que tiene los xml originales.

In [ ]:
import os
from os import path
import shutil

Source_Path = '/content/auto_annotate/temp'
Destination = '/content/auto_annotate/xml'
#dst_folder = os.mkdir(Destination)


def main():
    for count, filename in enumerate(os.listdir(Source_Path)):
        dst =  "image" + "_" + str(count+1) + ".jpg"

        # rename all the files
        os.rename(os.path.join(Source_Path, filename),  os.path.join(Destination, dst))


# Driver Code
if __name__ == '__main__':
    main()

se copian los xml y los jpg a una carpeta nueva en el Drive. Están listos para correr análisis con ellos.

In [ ]:
%cp -r '/content/auto_annotate/xml' '/content/drive/My Drive/training_images/1'

alternativamente, se puede devolver el script de abajo al original

In [ ]:
%rm - r /content/models/research/object_detection/utils/visualization_utils22.py
%mv "visualization_utils22.py" "visualization_utils.py"

# De aquí hacia abajo no usar, son solo pruebas

In [ ]:
!toco \
--graph_def_file='/content/test/tflite_graph.pb' \
--output_file='/content/test/detect.tflite' \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

este es floating point

In [ ]:
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
   --pipeline_config_path={pipeline_fname} \
  --output_directory={output_directory} \
  --trained_checkpoint_prefix={last_model_path}

In [ ]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path={pipeline_fname} \
--trained_checkpoint_prefix '/content/models/research/training/model.ckpt-1000' \
--output_directory={output_directory} \
--add_postprocessing_op=True 
--max_detections=10

In [ ]:
last_model_path = '/content/models/research/training'
print(last_model_path)

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path='/content/models/research/training/pipeline.config' \
--output_directory={output_directory} \
--trained_checkpoint_prefix='/content/models/research/training/model.ckpt-20000.index' \
--add_postprocessing_op=True \
--max_detections=10


In [ ]:
from google.colab import files
files.download(label_map_pbtxt_fname)

In [ ]:
files.download(pipeline_fname)

In [ ]:
# !tar cfz fine_tuned_model.tar.gz fine_tuned_model
# from google.colab import files
# files.download('fine_tuned_model.tar.gz')